In [13]:
import torch
from torch_geometric.nn import GraphConv, to_hetero
import torch 

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        self.conv = GraphConv(-1, hidden_channels)

    def forward(self, x, edge_index):

        x = self.conv(x, edge_index)
        return x

model = GCN( hidden_channels=5 )

In [19]:
from torch_geometric.data import HeteroData
from torch_geometric.utils import to_networkx


r"""
User(Bob) 0          User(Tom) 1 
          \         /       ↖
           \       /         \ 
            \     /           \
             ➘   ↙             ➘   
              Image <--------User  (Dan) 2 
                0 
"""

data                = HeteroData()

num_users           = 3
num_user_features   = 2
num_image_features  = 4
num_posts           = 1

data['user'].x  = torch.randn( num_users, num_user_features, dtype=torch.float )
data['image'].x = torch.randn( num_posts, num_image_features, dtype=torch.float )


data['user', 'follows', 'user'].edge_index = torch.tensor( [ [1], 
                                                             [2] ], dtype=torch.long )

data['user', 'post', 'image'].edge_index   = torch.tensor( [ [0], 
                                                             [0] ], dtype=torch.long )

data['user', 'likes', 'image'].edge_index  = torch.tensor( [ [1,2], 
                                                             [0,0] ], dtype=torch.long )


In [25]:
model = to_hetero(model, data.metadata())
print(f"Model is \n{model}")

# # out = model(data.x_dict, data.edge_index_dict)
# # print(f"Output is \n{out}")

# # print(f"\n\nWeights of the MODEL")
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model is 
GraphModule(
  (conv1): ModuleDict(
    (user__follows__user): ModuleDict(
      (user__follows__user): ModuleDict(
        (user__follows__user): HeteroConv(num_relations=3)
        (user__post__image): HeteroConv(num_relations=3)
        (user__likes__image): HeteroConv(num_relations=3)
      )
      (user__post__image): ModuleDict(
        (user__follows__user): HeteroConv(num_relations=3)
        (user__post__image): HeteroConv(num_relations=3)
        (user__likes__image): HeteroConv(num_relations=3)
      )
      (user__likes__image): ModuleDict(
        (user__follows__user): HeteroConv(num_relations=3)
        (user__post__image): HeteroConv(num_relations=3)
        (user__likes__image): HeteroConv(num_relations=3)
      )
    )
    (user__post__image): ModuleDict(
      (user__follows__user): ModuleDict(
        (user__follows__user): HeteroConv(num_relations=3)
        (user__post__image): HeteroConv(num_relations=3)
        (user__likes__image): HeteroConv(num_rela

In [27]:
from torch_geometric.nn import HeteroConv, PowerMeanAggregation

class HeteroGNN(torch.nn.Module): 
    def __init__(self, hidden_channels):
        super(HeteroGNN, self).__init__()
        self.hidden_channels = hidden_channels  
        # self.power_mean = PowerMeanAggregation(p=1.0, learn=True, channels=hidden_channels)
        aggr = ["sum"]
        self.conv1 = HeteroConv({
            ('user', 'follows', 'user'):    GraphConv(-1, hidden_channels, aggr = aggr),
            ('user', 'post', 'image'):      GraphConv(-1, hidden_channels, aggr = aggr),
            ('user', 'likes', 'image'):     GraphConv(-1, hidden_channels, aggr = aggr),
        }, aggr="cat")

    def forward(self, x_dict, edge_index_dict):
        print(f"Data shape is {x_dict['user'].shape}, {x_dict['image'].shape}")
        x_dict = self.conv1(x_dict, edge_index_dict)
        # print(f"After Conv shape is {x_dict['user'].shape}, {x_dict['image'].shape}")
        return x_dict
    
model = HeteroGNN(hidden_channels=5)
output = model(data.x_dict, data.edge_index_dict)
print(f"\n\nUser {output['user'].shape} | Image {output['image'].shape}")

HeteroGNN(
  (conv1): HeteroConv(num_relations=3)
)
Data shape is torch.Size([3, 2]), torch.Size([1, 4])


User torch.Size([3, 5]) | Image torch.Size([1, 10])


In [101]:
num_nodes       = 5
incoming_types  = 3
output_channels = 4

input = torch.randn(num_nodes, incoming_types, output_channels)
print(f"Input is {input.shape}")
input = input.view(num_nodes * incoming_types, output_channels)
print(f"Input reshaped is {input.shape}")

index = torch.arange(incoming_types).repeat_interleave(num_nodes)
print(f"Index is {index}, with shape {index.shape}")

power_mean = PowerMeanAggregation(p=1.0, learn=True, channels=output_channels)
output = power_mean(input, index)

print(f"\nOutput is {output.shape},\n{output}")


Input is torch.Size([5, 3, 4])
Input reshaped is torch.Size([15, 4])
Index is tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2]), with shape torch.Size([15])

Output is torch.Size([3, 4]),
tensor([[0.3324, 0.7316, 0.3480, 0.9962],
        [0.4709, 0.3610, 0.4245, 0.8073],
        [0.5580, 0.4791, 0.5209, 0.2184]], grad_fn=<PowBackward1>)


In [67]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define dimensions
num_nodes = 10
node_types = 5
output_channels = 16

# Example input tensor of shape [num_nodes, node_types, output_channels]

# Define the Attention-Based Pooling
class AttentionPooling(nn.Module):
    def __init__(self, node_types, output_channels, hidden_dim):
        super(AttentionPooling, self).__init__()
        
        # Linear transformation for attention computation
        self.attention_layer = nn.Sequential(
            nn.Linear(output_channels, hidden_dim),  # Transform output_channels to hidden_dim
            nn.ReLU(),  # Non-linearity
            nn.Linear(hidden_dim, 1)  # Scalar attention score for each node_type
        )
        
        # Optional softmax for normalization (across node_types)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        # x is of shape [num_nodes, node_types, output_channels]
        
        # Compute attention scores
        attention_scores = self.attention_layer(x)  # Shape: [num_nodes, node_types, 1]
        attention_scores = self.softmax(attention_scores)  # Normalize scores: [num_nodes, node_types, 1]
        
        # Multiply input by attention scores
        weighted_features = x * attention_scores  # Shape: [num_nodes, node_types, output_channels]
        
        # Aggregate across the node_types dimension
        aggregated_features = torch.sum(weighted_features, dim=1)  # Shape: [num_nodes, output_channels]
        
        return aggregated_features

# Instantiate the pooling layer
hidden_dim = 32  # Dimension of the hidden layer in the attention mechanism
attention_pooling = AttentionPooling(node_types=node_types, output_channels=output_channels, hidden_dim=hidden_dim)
input_tensor = torch.randn(13, node_types, output_channels)
print(f"Input tensor size is {input_tensor.shape}")

# Apply the pooling
pooled_output = attention_pooling(input_tensor)  # Shape: [num_nodes, output_channels]

# Print the result
print("Pooled output shape:", pooled_output.shape)

Input tensor size is torch.Size([13, 5, 16])
Pooled output shape: torch.Size([13, 16])


In [61]:
import torch
from torch_geometric.data import HeteroData
from torch_geometric.nn import HeteroConv, GraphConv

torch.manual_seed(12345)
# Create Heterogeneous Data
data = HeteroData()

num_users = 2
num_user_features = 1

data['user'].x = torch.randn(num_users, num_user_features, dtype=torch.float)
data['user', 'follows', 'user'].edge_index = torch.tensor([[0], [1]], dtype=torch.long)

# Define HeteroGNN
class HeteroGNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(HeteroGNN, self).__init__()
        aggr = ["sum"]
        self.conv1 = HeteroConv({
            ('user', 'follows', 'user'): GraphConv(-1, hidden_channels, aggr=aggr),
        }, aggr="sum")

        # Initialize placeholders for weights and biases
        self.rel_weight = None
        self.rel_bias = None
        self.root_weight = None

    def forward(self, x_dict, edge_index_dict):
        print(f"Input Features:\n{x_dict['user']}")

        # Perform HeteroConvolution
        x_dict = self.conv1(x_dict, edge_index_dict)

        # Extract and store weights and biases from GraphConv
        for edge_type, conv in self.conv1.convs.items():
            # print(f"Edge Type: {edge_type} | Conv: {conv}")

            # Store the weights and biases for later use
            self.rel_weight = conv.lin_rel.weight
            self.rel_bias = conv.lin_rel.bias
            self.root_weight = conv.lin_root.weight

        return x_dict


# Instantiate and run the model
model = HeteroGNN(hidden_channels=1)
output = model(data.x_dict, data.edge_index_dict)

# Display the output features after convolution
print(f"\nModel Output Features:\n{output['user']}")

# Aggregated feature calculation (for node 1, based on edge from node 0 -> node 1)
neighbor_feature = data['user'].x[0]  # Neighbor's feature (node 0)
aggregated_features = torch.matmul(neighbor_feature, model.rel_weight.T)  # Weighted aggregation
aggregated_features += model.rel_bias  # Add bias

print(f"\nChecking Manually for Node 1")
print(f"Aggregated Features (from neighbor): {aggregated_features.item()}")

# Self-loop calculation (for node 1)
self_feature = data['user'].x[1]  # Node 1's own feature
self_linear = torch.matmul(self_feature, model.root_weight.T)  # Self-loop transformation

print(f"Self Features (own transformation): {self_linear.item()}")

# Combine both to verify manual output
manual_output = aggregated_features + self_linear
print(f"Manual Output (Aggregated + Self): {manual_output.item()}")



Input Features:
tensor([[ 1.4271],
        [-1.8701]])

Model Output Features:
tensor([[-1.0215],
        [-1.3369]], grad_fn=<AddBackward0>)

Checking Manually for Node 1
Aggregated Features (from neighbor): -1.8327481746673584
Self Features (own transformation): 0.4958297610282898
Manual Output (Aggregated + Self): -1.3369183540344238


In [29]:
import torch
from torch_geometric.nn import GraphConv
from torch_geometric.data import Data
from torch_geometric.nn.aggr import MultiAggregation

torch.manual_seed(12345)    

# Define aggregation
aggr = ["sum", "mean"]
multi_aggr = MultiAggregation(aggr, mode="sum")

# Define the GraphConv layer
conv = GraphConv(in_channels=-1, out_channels=5, aggr=multi_aggr)

# Dummy input data
# Node feature matrix (4 nodes, 3 features per node)
x = torch.tensor([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0],
    [10.0, 11.0, 12.0]
], dtype=torch.float)

# Edge index (source, target pairs in COO format)
edge_index = torch.tensor([
    [0, 1, 2, 3, 0, 2],  # Source nodes
    [1, 0, 3, 2, 2, 0]   # Target nodes
], dtype=torch.long)

# Define graph data
data = Data(x=x, edge_index=edge_index)

# Apply the GraphConv layer
output = conv(data.x, data.edge_index)
print(conv.aggr)  # Check the aggregation method being used

print("\nOutput Node Features:")
print(output)
print(f"\nShape is ")
print(output.shape)


MultiAggregation([
  SumAggregation(),
  MeanAggregation(),
], mode=sum)

Output Node Features:
tensor([[ 28.8438, -18.1958,  -6.3359,   0.8343,  -6.0398],
        [  2.7006,  -3.5068,  -4.6705,  -0.5144,  -4.7905],
        [ 25.0298, -17.2939, -11.2107,  -0.9441,  -9.3804],
        [ 17.6409, -13.3752, -12.6950,  -2.4764,  -9.7636]],
       grad_fn=<AddBackward0>)

Shape is 
torch.Size([4, 5])
